In [0]:
%sql
SELECT crm_cust.Customer_ID ,
crm_cust.Customer_Key ,
crm_cust.Customer_first_name ,
crm_cust.Customer_last_name ,
crm_cust.marital_status ,
crm_cust.Gender ,
crm_cust.Customer_create_date ,
erp_cust.Date_Of_Birth


FROM bara_slaes_project.silver.customer_info AS crm_cust
LEFT JOIN  bara_slaes_project.silver.erp_customer_info AS erp_cust
ON crm_cust.Customer_ID = CAST(right(erp_cust.Customer_ID,5)AS INT);

In [0]:
query="""
SELECT 
    crm_cust.Customer_ID ,
    crm_cust.Customer_Key ,
    crm_cust.Customer_first_name ,
    crm_cust.Customer_last_name ,
    crm_cust.marital_status ,
    crm_cust.Gender ,
    crm_cust.Customer_create_date ,
    erp_cust.Date_Of_Birth

FROM bara_slaes_project.silver.customer_info AS crm_cust
LEFT JOIN  bara_slaes_project.silver.erp_customer_info AS erp_cust
ON crm_cust.Customer_ID = CAST(right(erp_cust.Customer_ID,5)AS INT);
"""

In [0]:
df = spark.sql(query)
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.gold.DimCustomer_info")